In [2]:
# Importanto biblioteca
import pandas as pd
from string import ascii_uppercase
import os

# ajutando para que o IDENT do corretor traga as informações de escola, serie, letra da turma e numero do aluno
def processar_ident(ident):
    ident_str = str(ident).zfill(6)
    
    tdc_escola = ident_str[:2].lstrip('0') or '0'
    serie = ident_str[2]
    letra = ascii_uppercase[int(ident_str[3])-1] if ident_str[3] != '0' else 'a'
    numero = ident_str[4:].lstrip('0') or '0'
    
    return tdc_escola, serie, letra, numero

# processamento do arquivo
def parse_arquivo(conteudo):
    blocos = conteudo.split('--------------------------') # identificador de final de bloco
    dados = []
    
    for bloco in blocos:
        linhas = [linha.strip() for linha in bloco.split('\n') if linha.strip()] # Ajuste para quebra de linha
        
        ident = next((linha.split(': ')[1] for linha in linhas if linha.startswith('IDENT.:')), None) # receber dados até ident
        if not ident or len(ident) != 6:
            continue
       
        # identificação de início de alternativas para gerar repetição 
        respostas_linhas = []
        for i, linha in enumerate(linhas):
            if 'Respostas (+ indica resposta correta):' in linha:
                respostas_linhas = linhas[i+1:]
                break
        
        # processando ident após o recebimento        
        tdc_escola, serie, letra, numero = processar_ident(ident)

        # obter número da alternativa e reposta
        for linha in respostas_linhas:
            if '.' in linha:
                partes = linha.split('.', 1)
                questao = partes[0].strip().zfill(2)
                
                # tratamento de respostas
                resposta = partes[1].strip().split()[0]
                
                # Converter respostas inválidas para "Em Branco"
                if 'Resposta inválida' in linha:
                    resposta = 'Em Branco'
                else:
                    # Remover marcador para utilizar apenas as letras das alternativas
                    resposta = resposta.replace('+', '').strip()
                
                dados.append([
                    f"{ident}{questao}",
                    ident,
                    tdc_escola,
                    serie,
                    letra,
                    numero,
                    int(questao),
                    resposta
                ])
    
    return pd.DataFrame(dados, columns=[
        'ID', 'ID_PROVA', 'TDC_ESCOLA_ID', 'NUMERO_SERIE',
        'LETRA_TURMA', 'NUMERO_ALUNO_TURMA', 'QUESTAO', 'RESPOSTA'
    ])

# Configurar caminho da pasata local
pasta = r'.\resultados escolas'

# Processar todos os arquivos
df_final = pd.DataFrame()
for arquivo in os.listdir(pasta):
    if arquivo.lower().endswith('.txt'):
        caminho_completo = os.path.join(pasta, arquivo)
        
        with open(caminho_completo, 'r', encoding='utf-8') as f:
            conteudo = f.read()
            df = parse_arquivo(conteudo)
            df_final = pd.concat([df_final, df])

# Ordenar e formatar
df_final = df_final.sort_values(['ID_PROVA', 'QUESTAO'])
df_final['QUESTAO'] = df_final['QUESTAO'].astype(int)

# Visualizando DataFrame
df_final

,ID,ID_PROVA,TDC_ESCOLA_ID,NUMERO_SERIE,LETRA_TURMA,NUMERO_ALUNO_TURMA,QUESTAO,RESPOSTA
0,01610101,016101,1,6,A,1,1,A
1,01610102,016101,1,6,A,1,2,E
2,01610103,016101,1,6,A,1,3,A
3,01610104,016101,1,6,A,1,4,D
4,01610105,016101,1,6,A,1,5,E
...,...,...,...,...,...,...,...,...
795,05633006,056330,5,6,C,30,6,C
796,05633007,056330,5,6,C,30,7,A
797,05633008,056330,5,6,C,30,8,B
798,05633009,056330,5,6,C,30,9,E
